# Modelo de la alarma

El paquete de _Python_ [pgmpy](http://pgmpy.org) proporciona un marco de trabajo para las redes bayesianas.

En primer lugar, importamos los mÃ³dulos necesarios:

In [1]:
import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos grÃ¡ficos de probabilidad
import pgmpy.factors.discrete as pgmf  # Tablas de probabilidades condicionales y
                                       # factores de probabilidad
import pgmpy.inference as pgmi  # Inferencia probabilÃ­stica exacta

Para definir la red bayesiana del modelo de la alarma visto en clase construimos primero el DAG asociado.

In [2]:
Modelo_alarma = pgmm.BayesianModel()
Modelo_alarma.add_nodes_from(['Robo', 'Terremoto', 'Alarma', 'Llamada', 'Noticia'])
Modelo_alarma.add_edges_from([('Robo', 'Alarma'),
                              ('Terremoto', 'Alarma'),
                              ('Alarma', 'Llamada'),
                              ('Terremoto', 'Noticia')])

TambiÃ©n se podrÃ­an haber proporcionado las aristas directamente al crear la instancia de la clase `BayesianModel`, en cuyo caso los vÃ©rtices se crean automÃ¡ticamente a partir de ellas.

In [3]:
Modelo_alarma = pgmm.BayesianModel([('Robo', 'Alarma'),
                                    ('Terremoto', 'Alarma'),
                                    ('Alarma', 'Llamada'),
                                    ('Terremoto', 'Noticia')])

Podemos comprobar cuÃ¡les son los vÃ©rtices y las aristas de la componente cualitativa de la red bayesiana.

In [4]:
Modelo_alarma.nodes()

NodeView(('Llamada', 'Robo', 'Terremoto', 'Alarma', 'Noticia'))

In [5]:
Modelo_alarma.edges()

#Una vez definido el grafo, definimos la relación entre nodos (aristas)

OutEdgeView([('Robo', 'Alarma'), ('Terremoto', 'Alarma'), ('Terremoto', 'Noticia'), ('Alarma', 'Llamada')])

A continuaciÃ³n, asociamos a cada variable de la red una distribuciÃ³n de probabilidad condicional. Hay que tener en cuenta que el paquete _pgmpy_ asume que los valores de una variable que puede tomar $n$ valores son los nÃºmeros de 0 a $n - 1$.

Para cada variable creamos una instancia de la clase `TabularCPD`, proporcionando:
* el nombre de la variable (atributo `variable`),
* la cantidad de valores (cardinalidad) que puede tomar (atributo `variable_card`),
* una lista de listas, conteniendo cada una de estas las probabilidades para un valor concreto, segÃºn los valores de los padres (atributo `values`, el valor se transforma a un array),
* una lista con los nombres de los padres (atributo `evidence`, valor `None` si no se proporciona),
* una lista con la cantidad de valores (cardinalidad) que puede tomar cada uno de los padres (atributo `evidence_card`, valor `None` si no se proporciona).

In [6]:
#Añadimos las probabilidades, el tercer parámetro es una lista de nodos padre con su cardinalidad, [Robo,Terremoto] Los valores metidos 
#los valores .XX son los valores de probabilidad para 00,01,10,11. Podemos ver la info en forma de tabla.
Alarma_CPD = pgmf.TabularCPD('Alarma', 2, [[.99, .1, .1, .01], [.01, .9, .9, .99]],
                             ['Robo', 'Terremoto'], [2, 2])
print(Alarma_CPD)

╒═══════════╤═════════════╤═════════════╤═════════════╤═════════════╕
│ Robo      │ Robo_0      │ Robo_0      │ Robo_1      │ Robo_1      │
├───────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ Terremoto │ Terremoto_0 │ Terremoto_1 │ Terremoto_0 │ Terremoto_1 │
├───────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ Alarma_0  │ 0.99        │ 0.1         │ 0.1         │ 0.01        │
├───────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ Alarma_1  │ 0.01        │ 0.9         │ 0.9         │ 0.99        │
╘═══════════╧═════════════╧═════════════╧═════════════╧═════════════╛


In [7]:
#Hacemos lo mismo con el resto de valores, robo no tiene padres
Robo_CPD = pgmf.TabularCPD('Robo', 2, [[.9, .1]])
print(Robo_CPD)

╒════════╤═════╕
│ Robo_0 │ 0.9 │
├────────┼─────┤
│ Robo_1 │ 0.1 │
╘════════╧═════╛


In [8]:
Terremoto_CPD = pgmf.TabularCPD('Terremoto', 2, [[.99, .01]])
print(Terremoto_CPD)

╒═════════════╤══════╕
│ Terremoto_0 │ 0.99 │
├─────────────┼──────┤
│ Terremoto_1 │ 0.01 │
╘═════════════╧══════╛


In [9]:
#el padre de noticia es terremoto, se lo definimos
Noticia_CPD = pgmf.TabularCPD('Noticia', 2, [[.999, .01], [.001, .99]], ['Terremoto'], [2])
print(Noticia_CPD)

╒═══════════╤═════════════╤═════════════╕
│ Terremoto │ Terremoto_0 │ Terremoto_1 │
├───────────┼─────────────┼─────────────┤
│ Noticia_0 │ 0.999       │ 0.01        │
├───────────┼─────────────┼─────────────┤
│ Noticia_1 │ 0.001       │ 0.99        │
╘═══════════╧═════════════╧═════════════╛


In [10]:
Llamada_CPD = pgmf.TabularCPD('Llamada', 2, [[.99, .05], [.01, .95]], ['Alarma'], [2])
print(Llamada_CPD)

╒═══════════╤══════════╤══════════╕
│ Alarma    │ Alarma_0 │ Alarma_1 │
├───────────┼──────────┼──────────┤
│ Llamada_0 │ 0.99     │ 0.05     │
├───────────┼──────────┼──────────┤
│ Llamada_1 │ 0.01     │ 0.95     │
╘═══════════╧══════════╧══════════╛


In [11]:
#Distribuciones de probabilidad condicional CPDS en inglés.
#Así lo añadimos al modelo
#AÃ±adimos los cpds al modelo
Modelo_alarma.add_cpds(Alarma_CPD, Robo_CPD, Terremoto_CPD, Noticia_CPD, Llamada_CPD)
# Los imprimimos:
for m in Modelo_alarma.cpds: print(m)

╒═══════════╤═════════════╤═════════════╤═════════════╤═════════════╕
│ Robo      │ Robo_0      │ Robo_0      │ Robo_1      │ Robo_1      │
├───────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ Terremoto │ Terremoto_0 │ Terremoto_1 │ Terremoto_0 │ Terremoto_1 │
├───────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ Alarma_0  │ 0.99        │ 0.1         │ 0.1         │ 0.01        │
├───────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ Alarma_1  │ 0.01        │ 0.9         │ 0.9         │ 0.99        │
╘═══════════╧═════════════╧═════════════╧═════════════╧═════════════╛
╒════════╤═════╕
│ Robo_0 │ 0.9 │
├────────┼─────┤
│ Robo_1 │ 0.1 │
╘════════╧═════╛
╒═════════════╤══════╕
│ Terremoto_0 │ 0.99 │
├─────────────┼──────┤
│ Terremoto_1 │ 0.01 │
╘═════════════╧══════╛
╒═══════════╤═════════════╤═════════════╕
│ Terremoto │ Terremoto_0 │ Terremoto_1 │
├───────────┼─────────────┼─────────────┤
│ Noticia_0 │ 0.999       │ 0.01        │
├─

### d-separaciÃ³n

Con el mÃ©todo `is_active_trail` podemos comprobar si una variable es __condicionalmente dependiente__ de otra, dada una evidencia.

In [12]:
print(Modelo_alarma.is_active_trail('Robo', 'Llamada'))
#Robo es dependiente de llamada conociendo alarma?
print(Modelo_alarma.is_active_trail('Robo', 'Llamada', 'Alarma'))
print(Modelo_alarma.is_active_trail('Alarma', 'Noticia'))
print(Modelo_alarma.is_active_trail('Alarma', 'Noticia', 'Terremoto'))
print(Modelo_alarma.is_active_trail('Robo', 'Terremoto'))
print(Modelo_alarma.is_active_trail('Robo', 'Terremoto', 'Alarma'))
#lista de variables de evidencia
print(Modelo_alarma.is_active_trail('Robo', 'Terremoto', ['Llamada', 'Noticia']))

True
False
True
False
False
True
True


Podemos consultar cuÃ¡les son las variables __condicionalmente dependientes__ de una determinada variable, dada una evidencia.

In [13]:
#Cuales son las dependiente de robo

print(Modelo_alarma.active_trail_nodes('Robo'))
#Dependientes de robo con evidencia de alarma
print(Modelo_alarma.active_trail_nodes('Robo', 'Alarma'))
print(Modelo_alarma.active_trail_nodes('Robo', 'Terremoto'))
#Dependientes de robo con evidencia de alarma y terremoto, lista de evidencias de entrada.
print(Modelo_alarma.active_trail_nodes('Robo', ['Alarma', 'Terremoto']))

{'Robo': {'Llamada', 'Robo', 'Alarma'}}
{'Robo': {'Robo', 'Terremoto', 'Noticia'}}
{'Robo': {'Llamada', 'Robo', 'Alarma'}}
{'Robo': {'Robo'}}


### Inferencia probabilÃ­stica exacta mediante eliminaciÃ³n de variables

A continuaciÃ³n vamos a calcular, usando el algoritmo de eliminaciÃ³n de variables, la distribuciÃ³n de probabilidad de la variable Robo, dado que sabemos que Watson ha llamado a Holmes y que en la radio no han dado ninguna noticia de que se haya producido un terremoto.

Los factores iniciales serÃ¡n las tablas de probabilidad condicionales de la red bayesiana, reducidas segÃºn la evidencia. Eliminaremos primero la variable Terremoto y despuÃ©s la variable Alarma. Finalmente, multiplicaremos los factores restantes y normalizaremos.

Los factores se implementan como instancias de la clase `DiscreteFactor`. Para crear una instancia de un factor hay que proporcionar:
1. El Ã¡mbito del factor como una lista de variables (atributo `variables`).
2. Las cardinalidades de las variables del Ã¡mbito, como una lista (atributo `cardinality`).
3. Los valores asociados a cada combinaciÃ³n de valores de las variables del Ã¡mbito, como una lista (atributo `values`). Para establecer la asociaciÃ³n debe tenerse en cuenta que los valores de las variables varÃ­an mÃ¡s rÃ¡pido desde la Ãºltima variable del Ã¡mbito hasta la primera.

TambiÃ©n se puede obtener un factor a partir de una instancia de la clase `TabularCPD` mediante el mÃ©todo `to_factor` de esta Ãºltima.

In [14]:
# Factores iniciales

# Para cada variable, creamos su factor

phi_R = Robo_CPD.to_factor()
print(phi_R.scope())
print(phi_R)

phi_T = Terremoto_CPD.to_factor()
print(phi_T.scope())
print(phi_T)

phi_A = Alarma_CPD.to_factor()
print(phi_A.scope())
print(phi_A)

phi_N = Noticia_CPD.to_factor()
phi_N.reduce([('Noticia', 0)])
print(phi_N.scope())
print(phi_N)

phi_Ll = Llamada_CPD.to_factor()
phi_Ll.reduce([('Llamada', 1)])
print(phi_Ll.scope())
print(phi_Ll)

#

['Robo']
╒════════╤═════════════╕
│ Robo   │   phi(Robo) │
╞════════╪═════════════╡
│ Robo_0 │      0.9000 │
├────────┼─────────────┤
│ Robo_1 │      0.1000 │
╘════════╧═════════════╛
['Terremoto']
╒═════════════╤══════════════════╕
│ Terremoto   │   phi(Terremoto) │
╞═════════════╪══════════════════╡
│ Terremoto_0 │           0.9900 │
├─────────────┼──────────────────┤
│ Terremoto_1 │           0.0100 │
╘═════════════╧══════════════════╛
['Alarma', 'Robo', 'Terremoto']
╒══════════╤════════╤═════════════╤══════════════════════════════╕
│ Alarma   │ Robo   │ Terremoto   │   phi(Alarma,Robo,Terremoto) │
╞══════════╪════════╪═════════════╪══════════════════════════════╡
│ Alarma_0 │ Robo_0 │ Terremoto_0 │                       0.9900 │
├──────────┼────────┼─────────────┼──────────────────────────────┤
│ Alarma_0 │ Robo_0 │ Terremoto_1 │                       0.1000 │
├──────────┼────────┼─────────────┼──────────────────────────────┤
│ Alarma_0 │ Robo_1 │ Terremoto_0 │                     

In [15]:
# Eliminamos la variable Terremoto

phi_noT = phi_T * phi_A * phi_N
phi_noT.marginalize(['Terremoto'])
print(phi_noT.scope())
print(phi_noT)

['Robo', 'Alarma']
╒════════╤══════════╤════════════════════╕
│ Robo   │ Alarma   │   phi(Robo,Alarma) │
╞════════╪══════════╪════════════════════╡
│ Robo_0 │ Alarma_0 │             0.9791 │
├────────┼──────────┼────────────────────┤
│ Robo_0 │ Alarma_1 │             0.0100 │
├────────┼──────────┼────────────────────┤
│ Robo_1 │ Alarma_0 │             0.0989 │
├────────┼──────────┼────────────────────┤
│ Robo_1 │ Alarma_1 │             0.8902 │
╘════════╧══════════╧════════════════════╛


In [16]:
# Eliminamos la variable Alarma

phi_noA = phi_Ll * phi_noT
phi_noA.marginalize(['Alarma'])
print(phi_noA.scope())
print(phi_noA)

['Robo']
╒════════╤═════════════╕
│ Robo   │   phi(Robo) │
╞════════╪═════════════╡
│ Robo_0 │      0.0193 │
├────────┼─────────────┤
│ Robo_1 │      0.8467 │
╘════════╧═════════════╛


In [17]:
# Multiplicamos los factores restantes y normalizamos

phi = phi_R * phi_noA
phi.normalize()
print(phi)

╒════════╤═════════════╕
│ Robo   │   phi(Robo) │
╞════════╪═════════════╡
│ Robo_0 │      0.1700 │
├────────┼─────────────┤
│ Robo_1 │      0.8300 │
╘════════╧═════════════╛


Todos los pasos anteriores constituyen el algoritmo de eliminaciÃ³n de variables, que, creando instancias de la clase `VariableElimination`, se puede aplicar de manera automÃ¡tica para realizar consultas en una red bayesiana.

Una vez creada la instancia del algoritmo para una red bayesiana concreta, para realizar una consulta probabilÃ­stica hay que proporcionar los siguientes argumentos:
* Una lista de las variables de consulta.
* Un diccionario en el que se indique el valor conocido de cada variable de evidencia.
* Opcionalmente, una lista del resto de variables indicando el orden de eliminaciÃ³n. Si no se proporciona, este Ãºltimo se determina automÃ¡ticamente.

In [18]:
Modelo_alarma_ev = pgmi.VariableElimination(Modelo_alarma)
consulta = Modelo_alarma_ev.query(['Robo'], {'Llamada': 1, 'Noticia': 0},
                                  ['Terremoto', 'Alarma'])
print(consulta['Robo'])

#Tiene toda la pinta de que ha habido un robo

╒════════╤═════════════╕
│ Robo   │   phi(Robo) │
╞════════╪═════════════╡
│ Robo_0 │      0.1700 │
├────────┼─────────────┤
│ Robo_1 │      0.8300 │
╘════════╧═════════════╛


# Inferencia probabilÃ­stica aproximada

Una _muestra aleatoria_ de una red bayesiana es una asignaciÃ³n de valores a las variables de la red, con probabilidad de generaciÃ³n igual a la probabilidad conjunta de que las variables tomen esos valores.

__Ejercicio 1__: definir una funciÃ³n `muestra_aleatoria` que, dada una red bayesiana, devuelva una muestra aleatoria de la misma (como un diccionario).

__Notas__:
1. La funciÃ³n `topological_sort` del paquete `networkx` devuelve un orden topolÃ³gico de los vÃ©rtices de un DAG.
2. El mÃ©todo `get_cpds` de la clase `BayesianModel` devuelve la instancia de la clase `TabularCPD` asociada a la variable proporcionada como argumento.
3. Dados una instancia de la clase `TabularCPD`, un entero representando un valor _Val_ de la variable _Var_ asociada a ella y un diccionario representando una asignaciÃ³n de valores a variables, incluidos los padres de _Var_, la funciÃ³n `seleccionar_probabilidad` definida a continuaciÃ³n devuelve la probabilidad de que _Var_ tome el valor _Val_, condicionada a los valores de los padres de _Var_.
4. Dados la cardinalidad _Card_ de una variable y una lista de probabilidades para cada entero entre $0$ y _Card_$ - 1$, la funciÃ³n `generar_valor_aleatorio` definida a continuaciÃ³n devuelve un entero aleatorio en ese rango generado segÃºn la probabilidad proporcionada.

In [19]:
import random

def seleccionar_probabilidad(cpd, valor, evidencia):
    padres = [v for v in cpd.variables if v != cpd.variable]
    valores_evidencia = tuple(evidencia[var] for var in padres)
    return cpd.values[valor][valores_evidencia]

def generar_valor_aleatorio(cardinalidad, probabilidades):
    p = random.random()
    acumuladas = 0
    for valor in range(cardinalidad):
        acumuladas += probabilidades[valor]
        if p <= acumuladas:
            return valor

**Muestra aleatoria**: asignaciÃ³n de valores a las variables, con probabilidad de generaciÃ³n segÃºn las probabilidades de la red.
* $ð‘‹_{1}, ..., ð‘‹_{ð‘›}$ orden topolÃ³gico de las variables
* para $ð‘–$ desde $1$ hasta $ð‘›$
    * $ð‘¥_{ð‘–} \leftarrow \mathrm{Val}(ð‘‹_{ð‘–})$ segÃºn la distribuciÃ³n $\mathbb{P}(ð‘‹_{ð‘–} | \mathrm{pa}(ð‘‹_{ð‘–}))$
    > Elegir un valor del dominio de $X_{i}$ teniendo en cuenta su distribuciÃ³n de probabilidad y que {$X_{1}: x_{1}, ..., X_{i-1}: x_{i-1}$}
* fin para
* devolver {$X_{1}: ð‘¥_{1}, ..., X_{n}: ð‘¥_{ð‘›}$}

In [20]:
def muestra_aleatoria(red):
    variables = networkx.topological_sort(red)
    muestra = {}
    for variables in variables:
        cpd = red.get_cpds(variables)
        probabilidades = [seleccionar_probabilidad(cpd,val,muestra)
                         for val in range(cpd.variable_card)]
        muestra[variable] = generar_valor_aleatorio(cpd,variable_card,probabilidades)
        return muestra

    random.seed(12345)
    muestra_aleatoria(Modelo_alarma)
    

Dada una evidencia, una _muestra ponderada_ de una red bayesiana es una muestra aleatoria en la que los valores asignados a las variables son compatibles con la evidencia, junto con el peso asociado a esa muestra, es decir, la probabilidad de que se tenga la evidencia dados los valores generados del resto de variables.

**Muestra aleatoria ponderada**: generaciÃ³n compatible con la evidencia $Îµ$:
* $ð‘‹_{1}, ..., ð‘‹_{ð‘›}$ orden topolÃ³gico de las variables
* $ð‘¤ \leftarrow 1$
* para $ð‘–$ desde $1$ hasta $ð‘›$
    * si $ð‘‹_{ð‘–}$ es de evidencia entonces
        * $ð‘¥_{ð‘–} \leftarrow Îµ[ð‘‹_{ð‘–}]$
        * multiplicar $ð‘¤$ por $\mathbb{P}(ð‘¥_{ð‘–} | \mathrm{pa}(ð‘‹_{ð‘–}))$
    * si no entonces
        * $ð‘¥_{ð‘–} \leftarrow \mathrm{Val}(ð‘‹_{ð‘–})$ segÃºn la distribuciÃ³n $\mathbb{P}(ð‘‹_{ð‘–} | \mathrm{pa}(ð‘‹_{ð‘–}))$
    * fin si
* fin para
* devolver ({$X_{1}: ð‘¥_{1}, ..., X_{n}: ð‘¥_{ð‘›}$}, $ð‘¤$)


__Ejercicio 2__: definir una funciÃ³n `muestra_ponderada` que, dados una red bayesiana y una evidencia (como un diccionario que asigna valores a variables), devuelva una tupla cuyo primer elemento sea una muestra aleatoria de la red (como un diccionario) y cuyo segundo elemento sea el peso de la misma (como un nÃºmero real).

In [21]:
def muestra_ponderada(red,evidencia):
    pass

__Ejercicio 3__: definir una funciÃ³n `muestreo_con_rechazo` que, dados una red bayesiana, unas variables de consulta (como una lista), una evidencia (como un diccionario) y un entero positivo $N$, devuelva la distribuciÃ³n de probabilidad conjunta (como una instancia de la clase `DiscreteFactor`) de las variables de consulta dada la evidencia, calculada mediante el algoritmo de muestreo con rechazo generando $N$ muestras aleatorias.

__Notas__:
1. ConsidÃ©rese el uso de la funciÃ³n `product` de la biblioteca estÃ¡ndar `itertools` para generar todas las combinaciones de valores de las variables de consulta.
2. ConsidÃ©rese el uso del tipo de dato `Counter` de la biblioteca estÃ¡ndar `collections` para llevar el conteo de las frecuencias con las que han aparecido cada una de esas combinaciones en las muestras no rechazadas.

**Algoritmo de muestreo con rechazo**
* $Ï• \leftarrow$ potencial de Ã¡mbito $ð’€$ que asigna $0$ a cada combinaciÃ³n de valores
> $Ï• \leftarrow$ asociar 0 a cada combinaciÃ³n de valores para las variables de consulta
* Generar $ð‘$ muestras aleatorias $ð‘’_{1}, \dotsc, ð‘’_{ð‘}$
> funciÃ³n `muestra_aleatoria` anterior
* para cada muestra $ð‘’$ generada hacer
    * si $ð‘’$ es compatible con $Îµ$ entonces
        * incrementar $Ï•(ð‘’[ð’€])$ en 1
        > modificar $Ï•$ incrementando en 1 el valor asociado a la combinaciÃ³n de valores para las variables de consulta que aparece en $ð‘’$
    * si no entonces
        * rechazar $ð‘’$
    * fin si
* fin para
* devolver $Î·(Ï•)$
> creamos el factor a partir $Ï•$ y normalizamos

In [22]:
import itertools
import collections

def muestreo_con_rechazo(red,consulta,evidencia,n):
    pass

__Ejercicio 4__: definir una funciÃ³n `ponderaciÃ³n_por_verosimilitud` que, dados una red bayesiana, unas variables de consulta (como una lista), una evidencia (como un diccionario) y un entero positivo $N$, devuelva la distribuciÃ³n de probabilidad conjunta (como una instancia de la clase `DiscreteFactor`) de las variables de consulta dada la evidencia, calculada mediante el algoritmo de ponderaciÃ³n por verosimilitud generando $N$ muestras aleatorias.

**Algoritmo de ponderaciÃ³n por verosimilitud**
* $Ï• \leftarrow$ potencial de Ã¡mbito $ð’€$ que asigna $0$ a cada combinaciÃ³n de valores
* Generar $ð‘$ muestras aleatorias ponderadas $(ð‘’_{1}, ð‘¤_{1}), ..., (ð‘’_{ð‘}, ð‘¤_{ð‘})$ compatibles con la evidencia
> funciÃ³n `muestra_ponderada` anterior
* para cada muestra $ð‘’_{ð‘–}$ generada hacer
    * sumar $ð‘¤_{ð‘–}$ a $Ï•(ð‘’_{ð‘–}[ð’€])$
    > modificar $Ï•$ incrementando en $ð‘¤_{ð‘–}$ el valor asociado a la combinaciÃ³n de valores para las variables de consulta que aparece en $ð‘’$
* fin para
* devolver $Î·(Ï•)$

In [23]:
def ponderacion_por_verosimilitud(red,consulta,evidencia,n):
    pass

# EstimaciÃ³n del riesgo de una aseguradora de coches

La siguiente red bayesiana modeliza un sistema experto para la estimaciÃ³n del riesgo de una asguradora de coches.

<img src='insurance.png' width=700 height=463>

La red bayesiana se puede leer desde un fichero en formato BIF evaluando las siguientes expresiones:

In [24]:
import pgmpy.readwrite.BIF as rwBIF

reader = rwBIF.BIFReader('insurance.bif')
Modelo_aseguradora = reader.get_model()

Las variables se clasifican en tres tipos: variables de interÃ©s, variables observables y variables mediadoras.

__Nota__: en lo que sigue se indica entre parÃ©ntesis el nombre de la variable tal y como aparece en el fichero, en aquellos casos en los que difiere del nombre que aparece en el grÃ¡fico.

Las variables de interÃ©s son: MedicalCost (`MedCost`, coste mÃ©dico), LiabilityCost (`ILiCost`, coste de responsabilidad), PropertyCost (`PropCost`, coste de propiedad). Las tres variables pueden tomar como valores: miles (`Thousand`), diez miles (`TenThou`), cientos de miles (`HundredThou`) y millones (`Million`).

Las variables observables son:
* Age, con valores: `Adolescent`, `Adult`, `Senior`.
* Mileage, con valores: `FiveThou`, `TwentyThou`, `FiftyThou`, `Domino`.
* VehicleYear, con valores: `Current`, `Older`.
* DrivingHist (`DrivHist`), con valores: `Zero`, `One`, `Many`.
* DrivQuality, con valores: `Poor`, `Normal`, `Excellent`.
* MakeModel, con valores: `SportsCar`, `Economy`, `FamilySedan`, `Luxury`, `SuperLuxury`.
* HomeBase, con valores: `Secure`, `City`, `Suburb`, `Rural`.
* GoodStudent, ExtraCar (`OtherCar`), SeniorTrain, Antilock y Airbag con valores: `True`, `False`.

El resto de variables son variables mediadoras.

__Ejercicio 5__: definir una funciÃ³n `estimaciÃ³n_de_costes` que, dada una evidencia de una o mÃ¡s variables observables (como un diccionario), devuelva el valor mÃ¡s probable de cada variable de interÃ©s (como un diccionario).

In [25]:
def estimacion_de_costes(evidencia):
    pass

In [26]:
estimacion_de_costes({'Age': 0,
                      'Mileage': 3,
                      'VehicleYear': 1,
                      'DrivHist': 0,
                      'DrivQuality': 0,
                      'MakeModel': 0,
                      'HomeBase': 0,
                      'GoodStudent': 0})